## Code of America NDoCH 2020 - Asset Map
### Code for Sacramento

## Change Log
* 08-21-2020: Baseline Version v0.1

### Introduction

This project focuses on the asset map item for the [CFA NDoCH](2) event; it consists of a web application to visualize various assets within the Sacramento area. The data is sourced from various open data portals.

Data processing and maps are created using Jupyter Notebook due to its ability to visualize results effectively and efficiently. Listed below are installation instructions and more about Jupyter and the Python programming language.

## Jupyter Installation
1. Download and install Jupyter Notebook from their [website](3)
2. Verify that Jupyter Notebook was installed and visible from Windows Start menu
3. Start Jupyter Notebook; it will start CMD shell and load in the web browser
4. Save this notebook and CSV data to your "Documents" folder and navigate to it from the Notebook start page
5. Open this notebook from the start page; file and cells should be viewable

## Jupyter Introduction
This notebook will require some basic understanding of the Python programming language, Jupyter platform and data analysis concepts.

Jupyter is a powerful collaborative tool which is open-source and light-weight. It provides all the tools necessary to run data analysis, visualization, statistics and data science [out of the box][4]. In addition, it has gain acceptance from industry and academia for collaborating on projects and publishing work.

Jupyter is a combination of text and code with the programming run-time built into the platform so there is no need to install additional software. The text is in the markdown file format (similar to HTML), and code in several languages. It is organized by cells which can consist of either text or code; placed together, they can be sent as a single document to share/publish work.

## Jupyter Notebook
Notebooks are organized by cells, which mainly consist of text (in markdown) and code (Python). It operations like a hybrid between MS Word and Excel file; whereas the entire file is like a document, the cells operate like a spreadsheet. For getting started, feel free to scroll down each cell and navigate around the cells for a quick tour. Here is a breakdown of how to view/edit cells:

1. Each cell may be edited by hitting ENTER; toggle between cells using the arrow keys or mouse/scroller
2. When editing a cell, be sure to select "markdown" for text or "code" before writing into it
3. Each cell can be run by hitting CTRL + ENTER or the "run" button form the menu bar
4. Output from each cell will appear below; if an error occurs, please read and try to debug it(!)
5. File can be saved by hitting CTRL + "s" or file/save from the pulldown menu above

## Quick Start
This notebook will require some Pythong programming, which is widely used and gain enough traction to be taught in [high school][5] and AP Computer Science [courses][6].

[Jupyter][7] supports several different languages (R, Scala and Julia); however, Python is the most popular of them and can be used for other tasks, primarily data science and web programming.

## Exercises
If you are new to Jupyter, then please review the links below:
1. [Intro Guide from DataQuest][8]
2. [Intro Guide from DataCamp][9]
3. [Notebook Intro from Medium][10]
4. [Data Science Tutorial][11]

If you are new to programming or Python, then please review the links below:
1. [Quick Start][12]
2. [Intro Tutorials][13]
3. [Free Code Camp Guide][14]

If you are new to programming or Markdown, then please review the links below:
1. [Quick Start from Github][15]
2. [Quick Start Guide][16]
3. [Quick Start Tutorial][17]

[2]: https://www.codeforamerica.org/events/national-day-of-civic-hacking-2020
[3]: https://jupyter.org/install
[4]: https://jupyter.org/jupyter-book/01/what-is-data-science.html
[5]: https://codehs.com/info/curriculum/intropython
[6]: https://code.org/educate/curriculum/high-school
[7]: https://jupyter.org/
[8]: https://www.dataquest.io/blog/jupyter-notebook-tutorial/
[9]: https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook
[10]: https://towardsdatascience.com/a-beginners-tutorial-to-jupyter-notebooks-1b2f8705888a
[11]: https://jupyter.org/jupyter-book/01/what-is-data-science.html
[12]: https://www.python.org/about/gettingstarted/
[13]: https://realpython.com/learning-paths/python3-introduction/
[14]: https://guide.freecodecamp.org/python/
[15]: https://guides.github.com/features/mastering-markdown/
[16]: https://www.markdownguide.org/getting-started/
[17]: https://www.markdowntutorial.com/


In [1]:
# 01 - load modules into notebook

# data analysis module
import pandas as pd

# numerical data modules
import numpy as np
import scipy

# data visualization module
import matplotlib.pyplot as plt

# adjust plot settings
%matplotlib inline

# data visualization module
# https://seaborn.pydata.org/
# import seaborn as sns; sns.set(color_codes=True)

# geospatial modules
from shapely.geometry import Point, Polygon
from shapely.geometry import shape, LineString, Point
import geopandas as gpd
import geojsonio
from descartes import PolygonPatch
import fiona

# geospatial and geojson modules
import folium
import os
import json

# install pip package in current kernel; run only for initial install:
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install seaborn==0.9.0


In [2]:
# 02.01 - data import

# note: module based on tutorial below
# https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/
# https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

# function to read csv file
# https://stackoverflow.com/questions/32400867/pandas-read-csv-from-url/41880513#41880513
def read_data(path):
    df = pd.read_csv(path)
    return(df)

# function to output csv file
def output_result(df, filepath):
    df.to_csv(filepath)

# note: reduce original file (500mb) by subset first 10k rows and replace file
# https://datacarpentry.org/python-ecology-lesson/03-index-slice-subset/index.html
# df_data = read_data("data/sfpd_report_2003-18.csv")
# df_data = df_data[0:10000]
# output_result(df_data, "data/sfpd_report_2003-18.csv")

# read in reduced file after processing steps above
df_data = read_data("data/sfpd_report_2003-18.csv")

# function to show table info
def data_profile(df, msg):
    # pass in variable into string
    # https://stackoverflow.com/questions/2960772/how-do-i-put-a-variable-inside-a-string
    print('*** Table Info: %s ***' % msg, '\n')
    print(df.info(), '\n')
    print('*** Table Info: Table Dimensions ***', '\n')
    print(df.shape, '\n')

data_profile(df_data, 'SFPD Reports')

# function to show unique value for given column
def show_unique(df, col):
    # pass in variable into string
    # https://stackoverflow.com/questions/2960772/how-do-i-put-a-variable-inside-a-string
    print('*** Unique Values: (%s) ***' % col, '\n')
    print(df[col].unique(), '\n')

# function to output summary stats
def summary_stats(df, col):
    # pass in variable into string
    # https://stackoverflow.com/questions/2960772/how-do-i-put-a-variable-inside-a-string
    print('*** Summary Stats: (%s) ***' % col, '\n')
    print(df[col].describe(), '\n')
    # print(col.describe())

# function to rename columns
# https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
def rename_col(df, old_col, new_col):
    df.rename(
        columns={old_col:new_col},
        inplace=True
    )
    return df


*** Table Info: SFPD Reports *** 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Unnamed: 0                                                10000 non-null  int64  
 1   PdId                                                      10000 non-null  int64  
 2   IncidntNum                                                10000 non-null  int64  
 3   Incident Code                                             10000 non-null  int64  
 4   Category                                                  10000 non-null  object 
 5   Descript                                                  10000 non-null  object 
 6   DayOfWeek                                                 10000 non-null  object 
 7   Date                                                      10000 non-n

In [3]:
# 02.02 - data clean

# function convert col to numeric type
# reference: https://stackoverflow.com/questions/47333227/pandas-valueerror-cannot-convert-float-nan-to-integer
def convert_num(df, col):
    # convert type
    df[col] = pd.to_numeric(
        df[col],
        errors='coerce'
    )
    return(df)

# convert string to datetime
# reference: https://stackoverflow.com/questions/32888124/pandas-out-of-bounds-nanosecond-timestamp-after-offset-rollforward-plus-adding-a
def convert_date(df, col):
    # convert type
    df[col] = pd.to_datetime(
        df[col],
        infer_datetime_format=True,
        errors = 'coerce'
    )
    return(df)

# function convert col to string type
def convert_str(df, col):
    # convert type
    df[col].astype(str)
    return(df)


In [11]:
# 03.01 - map plot

# note: module based on tutorial below
# https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/
# https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

# set map origin
SF_COORDINATES = (37.76, -122.45)

# for speed purposes
MAX_RECORDS = 1000

# create empty map zoomed in on San Francisco
map = folium.Map(location=SF_COORDINATES, zoom_start=16)

# add a marker for every record in the filtered data, use a clustered view
for each in df_data[0:MAX_RECORDS].iterrows():
    # note: adjust map settings and update syntax:
    # https://python-visualization.github.io/folium/quickstart.html
    folium.Marker(
        location = [each[1]['Y'],each[1]['X']],
        clustered_marker = True,
        popup = 'SFPD Crime Report'
    ).add_to(map)

display(map)
